In [57]:
import json
import numpy as np

In [58]:
inverted_index = json.loads(open('./output/inverted_index.json').read())

In [59]:
# Unique documents in the corpus
docs = {doc for docs_freq in inverted_index.values() for doc in docs_freq}

# Calculate IDF for each term
idf = {term: np.log10(len(docs) / len(doc_freq)) for term, doc_freq in inverted_index.items()}

# Calculate TF for each term in each document
tf = {doc: {term: np.log10(1 + inverted_index[term].get(doc, 0)) for term in inverted_index.keys()} for doc in docs}

# Calculate TF-IDF for each term in each document
tf_idf_1 = {doc: {term: tf[doc][term] * idf[term] for term in inverted_index} for doc in docs}

In [ ]:
len(tf_idf_1['d259'])

12463

In [ ]:
# Precompute the number of documents and document frequency for each term
num_docs = len({doc for docs_freq in inverted_index.values() for doc in docs_freq})
total_term_freq = {term: len(docs_freq) for term, docs_freq in inverted_index.items()}

# Precompute IDF values
idf = {term: np.log10(num_docs / df) for term, df in total_term_freq.items()}

# Initialize TF-IDF dictionary
tf_idf_2 = {doc: {} for doc in docs}

# Compute TF and TF-IDF for each document
for term, docs_freq in inverted_index.items():
    idf_value = idf[term]
    for doc, freq in docs_freq.items():
        tf_value = np.log10(1 + freq)
        tf_idf_2[doc][term] = tf_value * idf_value



In [ ]:
len(tf_idf_2['d259'])

131